<a href="https://colab.research.google.com/github/BoukeliaAbdelbasset/NLPDeepQSAR/blob/master/MultiVecLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd /content/drive/My Drive/KhalidNew

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import biovec
import os
pv = biovec.models.ProtVec('allData.fasta', corpus_fname="corpus",n=3,size=1024,workers=1)
pv.save("model")
 

In [0]:
pip install biopython

In [0]:
import pandas as pd
import os
i = 1912226
for f in os.listdir("allData"):
  file=open("allData/"+f)
  file2 = open("Transformed/"+f,"w")
  for line in file:
    o = ">"+str(i)+"@"+line.split(';')[1]+""
    sequence = o+'\n'+line.split(';')[0]+'\n'
    file2.writelines(sequence)
    file2.flush()
    i = i+1

In [0]:
import os 
i = 0;
file2 = open('allData.fasta','w')
for line in open('allData.csv'):
  o = ">"+str(i)+"@"+line.split(';')[1]+""
  sequence = o+'\n'+line.split(';')[0]+'\n'
  file2.writelines(sequence)
  file2.flush()
  i = i+1

In [0]:
import sys
import pandas as pd
import biovec
import os
import numpy as np
from Bio import SeqIO
pv = biovec.models.load_protvec('model')
for f in os.listdir("Transformed"):
    file = open("VectorizedT/"+f,"w")
    for seq in SeqIO.parse("Transformed/"+f, "fasta"):
      if(len(seq.seq)>= 3):
          try:
            array1 = sum(pv.to_vecs(seq.seq))
            c = str(np.array(array1).tolist())
            c = c.replace("[","").replace("]","").replace(", ",";")
            file.write(seq.id.split("@")[1].replace(',','.')+";"+c)
            file.flush()
            file.write('\n')
          except:
            print("error")


In [0]:
import os
allData = open('allData.fasta','w')
for f in os.listdir('Transformed'):
  for line in open('Transformed/'+f):
    allData.writelines(line)
    allData.flush()

In [0]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
import sklearn as sk
import numpy
import os
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot) )
import numpy as np
def loss_new(y_true, y_pred,y_train):
    return 1 / coeff_determination(y_true, y_pred)
datasett=pd.read_csv('VectorizedT/IGC50Train.txt', delimiter=';')
X_train = datasett.iloc[:, 1:1025].values
y_train = datasett.iloc[:, 0].values
dataset = pd.read_csv('VectorizedT/IG50Test.txt',delimiter=';')
X_test = dataset.iloc[:, 1:1025].values
y_test = dataset.iloc[:, 0].values
from sklearn.preprocessing import StandardScaler
X_train = X_train.reshape(len(X_train),32,32,1)
X_test = X_test.reshape(len(X_test),32,32,1)
model= Sequential()
model.add(Conv2D(256,(2,2),padding='valid', activation='relu', input_shape=(32,32,1)))
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss="mse", optimizer='adam',metrics=[coeff_determination,"mae"])
model.summary()
model.fit(X_train,y_train,batch_size=len(X_train),epochs=10,verbose=0)
y_predtrain = model.predict(X_train)
y_pred = model.predict(X_test)
R2interne = r2_score(y_train,y_predtrain)
print("R2 intern: "+str(R2interne))
scores = r2_score(y_test, y_pred)
print("R2 extern:"+ str(scores))
R2all = (R2interne-scores)/R2interne
print("Report R2:"+str(R2all))
print("MAE :"+str(mean_absolute_error(y_test, y_pred)))
print("RMSE :"+str(np.sqrt(mean_squared_error(y_test, y_pred))))
ypredicted = model.predict(X_test)
print(ypredicted)

In [0]:
from sklearn.ensemble import RandomForestRegressor
datasett=pd.read_csv('VectorizedT/IGC50Train.txt', delimiter=';')
X_train = datasett.iloc[:, 1:1025].values
y_train = datasett.iloc[:, 0].values
dataset = pd.read_csv('VectorizedT/IG50Test.txt',delimiter=';')
X_test = dataset.iloc[:, 1:1025].values
y_test = dataset.iloc[:, 0].values
model= RandomForestRegressor(n_estimators=200, random_state=0)
model.fit(X_train,y_train)
y_predtrain = model.predict(X_train)
R2interne = r2_score(y_train,y_predtrain)
print("R2 intern: "+str(R2interne))
y_pred = model.predict(X_test)
scores = r2_score(y_test, y_pred)
print("R2 extern:"+ str(scores))
R2all = (R2interne-scores)/R2interne
print("Report R2:"+str(R2all))
print("MAE :"+str(mean_absolute_error(y_test, y_pred)))
print("MSE: "+str(mean_squared_error(y_test,y_pred)))


In [0]:
from sklearn.svm import SVR
datasett=pd.read_csv('VectorizedT/IGC50Train.txt', delimiter=';')
X_train = datasett.iloc[:, 1:1025].values
y_train = datasett.iloc[:, 0].values
dataset = pd.read_csv('VectorizedT/IG50Test.txt',delimiter=';')
X_test = dataset.iloc[:, 1:1025].values
y_test = dataset.iloc[:, 0].values
model= SVR(gamma='scale', C=30.0, epsilon=0.2,verbose=1)
model.fit(X_train,y_train)
y_predtrain = model.predict(X_train)
R2interne = r2_score(y_train,y_predtrain)
print("R2 intern: "+str(R2interne))
y_pred = model.predict(X_test)
scores = r2_score(y_test, y_pred)
print("R2 extern:"+ str(scores))
R2all = (R2interne-scores)/R2interne
print("Report R2:"+str(R2all))
print("MAE :"+str(mean_absolute_error(y_test, y_pred)))
print("MSE: "+str(mean_squared_error(y_test,y_pred)))